In [1]:
# using inception
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# prepare data

batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='data/mnist/', train=True, download=False, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='data/mnist/', train=False, download=False, transform=transform)
test_loader = DataLoader(test_dataset,shuffle=False, batch_size=batch_size)

#design model using class
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x+y)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(self.conv1(x))
        x = self.rblock1(x)
        x = self.mp(self.conv2(x))
        x = self.rblock2(x)
#         print(x.size())
        x = x.view(in_size, -1)
        x = self.fc(x)
        
        return x

model = Net()

# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# training cycle forward, backward, update
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on testset: %d %%' % (100*correct/total))

In [2]:
for epoch in range(10):
    train(epoch)
    test()

[1,   300] loss: 0.558
[1,   600] loss: 0.165
[1,   900] loss: 0.112
accuracy on testset: 97 %
[2,   300] loss: 0.089
[2,   600] loss: 0.077
[2,   900] loss: 0.070
accuracy on testset: 98 %
[3,   300] loss: 0.059
[3,   600] loss: 0.058
[3,   900] loss: 0.054
accuracy on testset: 98 %
[4,   300] loss: 0.048
[4,   600] loss: 0.043
[4,   900] loss: 0.046
accuracy on testset: 98 %
[5,   300] loss: 0.037
[5,   600] loss: 0.040
[5,   900] loss: 0.037
accuracy on testset: 98 %
[6,   300] loss: 0.034
[6,   600] loss: 0.033
[6,   900] loss: 0.033
accuracy on testset: 98 %
[7,   300] loss: 0.030
[7,   600] loss: 0.026
[7,   900] loss: 0.028
accuracy on testset: 98 %
[8,   300] loss: 0.023
[8,   600] loss: 0.027
[8,   900] loss: 0.027
accuracy on testset: 98 %
[9,   300] loss: 0.021
[9,   600] loss: 0.022
[9,   900] loss: 0.024
accuracy on testset: 98 %
[10,   300] loss: 0.018
[10,   600] loss: 0.021
[10,   900] loss: 0.021
accuracy on testset: 98 %
